In [2]:
# Import TensorFlow
import numpy as np
import tensorflow as tf

# Keras imports
from tensorflow.keras import datasets
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Custom imports
from kafnet import Kaf
import tqdm

In [3]:
# Load Breast Cancer dataset
(X_train, y_train), (X_test, y_test) = datasets.boston_housing.load_data()

In [16]:
# Initialize a KAF neural network
kafnet = Sequential([
    Dense(20, input_shape=(13,)),
    Kaf(20),
    Dense(1),
])

In [17]:
# Use tf.data DataLoader
train_data = tf.data.Dataset.from_tensor_slices((X_train.astype(np.float32), y_train.reshape(-1, 1)))
test_data = tf.data.Dataset.from_tensor_slices((X_test.astype(np.float32), y_test.astype(np.float32).reshape(-1, 1)))

In [18]:
from tensorflow.keras import losses, optimizers, metrics

opt = optimizers.Adam()

In [19]:

# training
for e in tqdm.trange(300, desc='Training'):

    for xb, yb in train_data.shuffle(1000).batch(32):

            with tf.GradientTape() as tape:
                loss = tf.losses.mean_squared_error(yb, kafnet(xb))
            g = tape.gradient(loss, kafnet.variables)
            opt.apply_gradients(zip(g, kafnet.variables))

Training: 100%|██████████| 300/300 [00:24<00:00, 12.10it/s]


In [20]:
# Evaluation
err = tf.metrics.Mean()
for xb, yb in test_data.batch(32):
    err((yb - kafnet(xb))**2)
print('Final error is: ' + str(err.result()))

Final error is: tf.Tensor(26.72754, shape=(), dtype=float32)
